In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'categorical', 'na_percentage': 0.0},
                'pir': {'type': 'categorical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [7]:
pm = PETsARD.HyperProcessor(metadata)

In [8]:
from PETsARD.Processor.Missingist import Missingist_Mean, Missingist_Drop
from PETsARD.Processor.Outlierist import Outlierist_IsolationForest

In [9]:
config_1 = {'missingist': {'gen': Missingist_Drop(),
  'age': Missingist_Mean()},
 'outlierist': {'gen': Outlierist_IsolationForest(),
  'age': None}}

In [10]:
pm.update_config(config_1)

In [11]:
pm._config

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152205990>,
  'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152205150>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x151fba6b0>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x151fbae00>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x151fba980>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x151fbb220>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1522046d0>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152205750>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152207220>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1522070d0>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152206fb0>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152206830>},
 'outlierist': {'gen': <PETsARD.Processor.Outlierist.Outlierist_IsolationForest at 0x1522074f0>,
  'age': None,
  'race':

In [12]:
pm.fit(df, ['missingist', 'encoder', 'scaler', 'outlierist'])

In [13]:
df

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,53.0,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,Male,78.0,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,Female,56.0,White,Graduate,Parther,42.400002,1,0,0.0,0.0,Q3,0
4,Female,42.0,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4185,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4186,Female,76.0,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4187,Female,80.0,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4188,Male,35.0,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [14]:
df_transformed = pm.transform(df)

In [15]:
df_transformed

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,0.634673,0.645459,0.100372,0.406771,0.067405,-0.202041,0.436993,0.093885,0.0,0.0,0.346324,0.936988
1,0.559860,0.142249,0.127102,0.558247,0.800471,0.235906,0.364833,0.900102,0.0,0.0,0.157193,0.397917
2,0.682514,1.540055,0.260577,0.562005,0.317490,-0.056059,0.653768,0.655433,0.0,0.0,0.849669,0.877001
3,0.322618,-0.472785,0.421770,0.137572,0.756882,-1.296911,0.821033,0.435126,0.0,0.0,0.582192,0.600155
4,0.367823,1.204581,0.694418,0.894508,0.989350,-0.085255,0.477440,0.490726,0.0,0.0,0.145048,0.474125
...,...,...,...,...,...,...,...,...,...,...,...,...
2368,0.200166,-0.528697,0.738477,0.450962,0.367386,-1.209322,0.134762,0.653936,0.0,0.0,0.000460,0.695051
2369,0.782819,0.142249,0.562606,0.973374,0.334151,0.411086,0.378527,0.923233,0.0,0.0,0.615675,0.977692
2370,0.504551,1.036845,0.624618,0.856239,0.142796,-0.158247,0.678989,0.897267,0.0,0.0,0.138847,0.799170
2371,0.714391,-1.423293,0.765124,0.324805,0.594430,-1.194723,0.590162,0.467522,0.0,0.0,0.157117,0.630549


In [16]:
df_inverse = pm.inverse_transform(df_transformed)

In [17]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.0,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,53.0,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,Male,78.0,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,Female,42.0,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
4,Female,72.0,Mexican,11th,Separated,28.600000,0,0,0.0,0.0,Q1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2368,Female,41.0,Other,Graduate,Married,20.900000,0,0,0.0,0.0,Q1,0
2369,Male,53.0,Mexican,11th,Married,32.000000,0,1,0.0,0.0,Q4,1
2370,Female,69.0,Mexican,9th,Married,28.100000,0,1,0.0,0.0,Q1,1
2371,Male,25.0,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0


In [18]:
(df_inverse == df).all()

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [ ]:
np.isclose(df_inverse.age, df.age).all()

True

In [ ]:
np.isclose(df_inverse.bmi, df.bmi).all()

True